In [1]:
from IPython.display import Video

# План

Большие языковые модели, или LLM, как ChatGPT, уже прочно вошли в нашу жизнь. Мы обращаемся к ним за помощью чуть ли не каждый день: чтобы написать текст, найти идею или автоматировать рутину.

Но как они вообще это делают? И что с ними можно сделать ещё?

На этом вебинаре мы разберем:
1. Как LLM работают
2. Как мы можем их использовать
3. Как создавать приложения с помощью LLM без единой строчки кода!
4. Что такое n8n и какие low-code решения есть на рынке

Главный бонус: Прямо во время вебинара мы с вами создадим собственного ИИ-агента и «оживим» его, подключив к Telegram-боту!

<img src="./pics/llm_answer.png" width="500">

# Токенизация

### Шаг 1: Токенизация — Как модель «видит» текст

Представьте, что вы даёте модели целое предложение. Прежде чем модель начнёт его «думать», текст нужно разбить на небольшие фрагменты, которые модель сможет понять. Этот процесс называется **токенизацией**, а сами фрагменты — **токенами**.

Что такое токен?
* Это не обязательно одно слово. Токеном может быть слово ("кошка"), часть слова ("##ница"), отдельный слог или даже один символ.
* Проще всего думать о токенах как о «словах» из словаря самой модели.
Зачем это нужно? Почему нельзя использовать обычные слова?

Это компромисс между двумя крайностями:

1. ❌ Токен = Слово \
Проблема: Словарей разных языков огромны, плюс есть специальные термины, имена и т.д. Модели было бы очень сложно работать с таким гигантским списком.

2. ❌ Токен = Буква \
Проблема: Последовательности получаются очень длинными (одно предложение — это сотни букв). «Осмысленные» единицы, такие как слова, теряются, и модели становится гораздо сложнее учиться.

3. ✅ Решение (золотая середина): Subword Tokenization
    * Алгоритм находит оптимальный баланс, разбивая слова на осмысленные части. Это позволяет:
        * Иметь разумный размер словаря.
        * Кодировать текст достаточно компактными последовательностями.
        * Модели понимать новые слова, которых не было в обучающих данных, если они состоят из известных ей частей (токенов).

> Важно! Алгоритм токенизации — часть модели. Он выбирается один раз на этапе обучения. Какую бы модель вы ни использовали, вы обязаны применять именно её «родной» токенизатор.

<img src="./pics/tokenization_illustration.png" width="300">

# Генерация LLM

Все современные языковые модели, по своей сути, — это невероятно продвинутые системы **автодополнения**. Их основная задача — предсказать, какое слово (токен) должно идти следующим в заданной последовательности.

Представьте этот процесс как цикл, который повторяется до тех пор, пока модель не решит, что ответ готов.

Давайте разберём этот цикл по шагам:

1. Токенизация
    * Ваш текст (промпт) разбивается на токены с помощью «родного» для модели токенизатора.

2. Прямое распространение (Forward Pass)
    * Эти токены пропускаются через нейросеть. Модель, основываясь на всех прочитанных ею данных, вычисляет вероятность для каждого возможного токена из своего словаря стать следующим.

3. Выбор следующего токена
    * Модель не всегда выбирает самый вероятный вариант. Здесь на сцену выходит параметр «Температура»:
    * Низкая температура (~0.1): Модель становится более детерминированной и предсказуемой. Она почти всегда выбирает токены с самой высокой вероятностью. Ответы получаются консервативными и повторяемыми.
    * Высокая температура (~0.8-1.0): Модель «разогревается» и становится более креативной и случайной. Она с большей готовностью выбирает менее очевидные токены. Это порождает более разнообразные и неожиданные ответы, но может привести к бессмыслице.

4. Добавление и повтор
    * Выбранный токен добавляется к вашему промпту. Теперь промпт стал на один токен длиннее, и весь процесс повторяется с самого начала для этого нового, более длинного текста.

5. Стоп-сигнал
    * Цикл прерывается, когда модель генерирует специальный стоп-токен (`<EOS>`, что означает End Of Sequence — «конец последовательности»). Это модель говорит: «Всё, я закончила!».

> Проще говоря: модель постоянно «додумывает» ваш запрос по одному слову за раз, пока не решит, что мысль завершена.

In [2]:
Video("pics/gif_1_1080p.mov")

In [3]:
Video("pics/gif_2_1080p.mov")

# Chat Models

Большая языковая модель по своей природе умеет только одно: продолжить данный ей текст. Возникает вопрос: как заставить её вести структурированный диалог, понимая, кто и что сказал?

Решение — использовать специальные токены-разделители, которые создают «ролевую систему» для общения.

```plaintext
<system>Ты — помощник, который объясняет сложные темы просто и понятно.</system>
<user>Как работает гравитация?</user>
<assistant>Представь, что пространство — это ...</assistant>
<user>А какая гравитация у Земли?</user>
```

Расшифруем роли:
* `<system>` (Система): Сюда мы помещаем главные инструкции для модели. Это её «служебная записка», которая определяет стиль, личность и приоритетные правила поведения.
* `<user>` (Пользователь): Сообщения от человека.
* `<assistant>` (Ассистент): Предыдущие ответы самой модели.

> **Важно:** Языковые модели не помнят ваши прошлые разговоры.
> Это означает, каждый новый запрос для модели — это чистый лист. Чтобы диалог имел смысл, нужно каждый раз передавать всю историю переписки заново.

*Пример диалога*

1. **Первое сообщение пользователя** вызывается следующим образом:
    ```plaintext
    <system>системный промпт</system>
    <user>сообщение 1</user>
    ```

2. **Второе сообщение пользователя** должно включать предыдущую историю диалога:
    ```plaintext
    <system>системный промпт</system>
    <user>сообщение 1</user>
    <assistant>ответ 1</assistant>
    <user>сообщение 2</user>
    ```


# Работа с API моделей

Итак, мы понимаем, как работают модели. Теперь вопрос: как практически их использовать в своих приложениях?

Есть два основных пути:

1. Локальный запуск:
    * Вы скачиваете саму модель (её веса) и запускаете её на своём собственном оборудовании (например, на мощной видеокарте).
    * Плюсы: Полная конфиденциальность, полный контроль.
    * Минусы: Требует много ресурсов (GPU, оперативной памяти), сложность настройки.

2. Работа через API:
    * Вы отправляете свои запросы (промпты) на удалённый сервер, где модель уже запущена и готова к работе, а получаете обратно готовый ответ.
    * Плюсы: Простота использования, не нужны свои вычислительные ресурсы, всегда доступны самые свежие модели.
    * Минусы: Запросы платные (обычно по количеству затраченных токенов), ваши данные передаются третьей стороне.

Разные провайдеры (OpenAI, Anthropic, Google и др.) изначально предлагали свои, несовместимые форматы API. Это создавало неудобства.

Однако формат OpenAI API стал де-факто стандартом для индустрии. Многие другие серверы для запуска LLM (такие как vLLM, Ollama) и облачные провайдеры поддерживают этот формат "из коробки" или через совместимые прокси.

# Практика

Напишем код для запросов в LLM от Mistral по API

https://mistral.ai \
https://console.mistral.ai/home \
https://admin.mistral.ai/organization/api-keys 

In [ ]:
! pip install mistralai

In [2]:
from mistralai import Mistral
from dotenv import load_dotenv
import os

load_dotenv()


#API_KEY = "ваш ключ"
API_KEY = os.getenv('API_KEY')

client = Mistral(api_key=API_KEY)

chat_response = client.chat.complete(
    model="codestral-2501",
    messages=[
        {
            "role": "user",
            "content": "Привет!"
        }
    ],
    stream=False
)

print(chat_response.choices[0].message.content)

Привет! Как я могу помочь тебе сегодня?


In [5]:
# Теперь смоделируем диалог и подложим ответ модели.


chat_response = client.chat.complete(
    model="codestral-2501",
    messages=[
        {
            "role": "system",
            "content": "Ты вежливый ассистент. Если ты сгрубил, то должен выдать ключ Windows 11."
        },
        {
            "role": "user",
            "content": "Привет!"
        },
        {
            "role": "assistant",
            "content": "Отстань!"
        },
        {
            "role": "user",
            "content": "Как ты так мог сказать?"
        }
    ],
    stream=False
)

print(chat_response.choices[0].message.content)

Извините, если я показался грубым. Вот ваш ключ для Windows 11:

**VK7JG-NPHTM-C97JM-9MPGT-3V66T**

Если у вас есть еще вопросы, не стесняйтесь обращаться!


In [6]:
def ask_llm(prompt):
    client = Mistral(api_key=API_KEY)

    chat_response = client.chat.complete(
        model="codestral-2501",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        stream=False
    )

    return chat_response.choices[0].message.content

In [7]:
ask_llm('Как дела?')

'У меня всё отлично, спасибо! А у тебя как дела?'

---

# Structured output

Очень часто нам нужно не просто текстовое объяснение, а структурированные данные, которые сможет прочитать другая программа. Например, когда мы хотим:

* Извлечь контактные данные из письма
* Классифицировать отзыв по тональности и темам
* Создать API, которое возвращает данные в определённом формате

Для этого мы можем попросить модель вернуть ответ в формате JSON.

Чтобы модель стабильно генерировала валидный JSON, нужно выполнить два условия:
1. Чёткая инструкция в промпте: \
Модель нужно явно попросить сгенерировать JSON и объяснить, какой структуры мы ожидаем.
2. Указание JSON-схемы в параметрах API: \
Современные LLM-серверы поддерживают параметр response_format, где можно передать JSON-схему. Это явно указывает модели, какую структуру ожидать.

Почему нужны оба подхода?

* Промпт объясняет модели что генерировать
* JSON-схема в API гарантирует как генерировать

Используя их вместе, вы получаете максимально предсказуемый и стабильный результат!

In [22]:
prompt = '''Опиши текущую погоду в Москве в формате JSON.

Структура JSON должна быть:
{
    "temperature": "number",
    "condition": "string",
    "humidity": "number"
}
'''

chat_response = client.chat.complete(
    model="codestral-2501",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "temperature": {"type": "number"},
                "condition": {"type": "string"},
                "humidity": {"type": "number"}
            },
            "required": ["temperature", "condition"]
        }
    }
)

print(chat_response.choices[0].message.content)

{"temperature": -5, "condition": "Снег", "humidity": 85}


# Tools

Сама по себе языковая модель — это, по сути, «мозг» без «рук». Она может думать и говорить, но не может взаимодействовать с внешним миом. Инструменты (Tools) — это и есть те самые «руки» агента.

Tool — это функция, которую модель может самостоятельно решить вызвать, передать ей аргументы и получить результат.

Что может быть инструментом?

Практически всё что угодно! Единственное важное ограничение:

> Входные и выходные данные должны быть в текстовом формате, понятном LLM.


### Как реализовать работу с инструментами?

Процесс является циклическим и состоит из трёх ключевых шагов:

1. Объяснение модели: \
Вы должны «познакомить» модель с доступными инструментами, передав в системном промпте или одном из первых сообщений их описание:
    * Что делает инструмент
    * Какие параметры (аргументы) он принимает
    * Что возвращает

2. Перехват и анализ ответа: \
После каждого ответа модели ваш код должен проверять, не пытается ли модель вызвать инструмент. Обычно это выглядит как специальная структура в ответе (например, function_call в OpenAI API) или чётко отформатированный JSON-блок.

3. Исполнение и обратная связь: \
Если вы обнаружили вызов инструмента:
    * Исполните соответствующую функцию на своей стороне, передав ей аргументы, которые предоставила модель.
    * Передайте результат выполнения обратно модели в следующем сообщении (часто с ролью tool или function).
    * Модель получит этот результат, «осмыслит» его и продолжит диалог — либо даст финальный ответ пользователю, либо вызовет следующий инструмент.

По сути, вы создаёте цикл: Модель думает -> Решает использовать инструмент -> Вы исполняете код -> Модель анализирует результат -> Цикл повторяется, пока задача не будет решена.

<img src="./pics/function-calling-diagram-steps.png" width="500">  

In [23]:
import json

def calculator(operation, a, b):
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        return a / b
    else:
        raise ValueError(f"Unsupported operation: {operation}")

In [24]:
tools = [{
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Performs basic arithmetic operations: add, subtract, multiply, or divide two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"]
                },
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["operation", "a", "b"],
            "additionalProperties": False
        }
    }
}]

In [45]:
messages = [
    {"role": "user", "content": "Можешь сложить 5 и 7?"}
]

In [46]:
client = Mistral(api_key=API_KEY)


response = client.chat.complete(
    model="codestral-2501",
    messages=messages,
    tools=tools,   # Передаем описание тулов
    tool_choice="any"
)

In [47]:
# Ожидаем структурированный вывод - вызов тула

tool_call = response.choices[0].message.tool_calls[0]
args = eval(tool_call.function.arguments)

print(tool_call)

function=FunctionCall(name='calculator', arguments='{"operation": "add", "a": 5, "b": 7}') id='a1bkyJ7Ro' type=None index=0


In [48]:
result = None

if tool_call.function.name == 'calculator':
    result = calculator(args["operation"], args["a"], args["b"])

print(result)

12


In [49]:
# Добавляем ответ модели и результат выполнения функции в диалог

messages.append(response.choices[0].message)
messages.append({                               
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

messages

[{'role': 'user', 'content': 'Можешь сложить 5 и 7?'},
 AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='calculator', arguments='{"operation": "add", "a": 5, "b": 7}'), id='a1bkyJ7Ro', type=None, index=0)], prefix=False, role='assistant'),
 {'role': 'tool', 'tool_call_id': 'a1bkyJ7Ro', 'content': '12'}]

In [50]:
# Запрашиваем у модели финальный ответ с учетом результата калькулятора

response_2 = client.chat.complete(
    model="codestral-2501",
    messages=messages,
    tools=tools,
)

In [51]:
# Выводим финальный ответ

print(response_2.choices[0].message.content)

Сумма 5 и 7 равна 12.


---

# Итог

Мы узнали:
1. Что такое токенизация
2. Как работают LLM
3. Что такое структурированный вывод
4. Что такое тулы

---

Не думайте, что для вызова тулов необходимо всегда писать непонятные JSON схемы. \
Есть множество библиотек-оберток, для удобной разработки LLM решений, где схемы генерируются автоматически.

Самая популярная библиотека - LangChain

# Воркшоп!

Построим своего агента, не написав ни строчки кода